In [1]:
import cryo_em_select as cryo


In [5]:
from pathlib import Path
import os

data_path = [x for x in Path(str(os.getcwd()) + '/data_example/raw_data/').iterdir()]
test = cryo.CryoBatchGenerator(data_path, batch_size=2)
test.__getitem__(1)
test.__len__()

[WindowsPath('c:/Users/Joachim Brendborg/Documents/Universitet/7. Semester/Deep Learning for Visual Recognition/cryo-em_segmentation/data_example/raw_data/May08_03.15.02.bin_avg.png'), WindowsPath('c:/Users/Joachim Brendborg/Documents/Universitet/7. Semester/Deep Learning for Visual Recognition/cryo-em_segmentation/data_example/raw_data/May08_03.43.44.bin_avg.png')]


9

In [3]:
import os
print(os.getcwd())

c:\Users\Joachim Brendborg\Documents\Universitet\7. Semester\Deep Learning for Visual Recognition\cryo-em_segmentation
